In [1]:
import torch 
import numpy as np
import time
import os
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [2]:
start_time=time.time()
#operations related to matrices
zeroes=torch.zeros(1,1)
end_time=time.time()
end_time-start_time

0.0

In [3]:
with open('wiz_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()

In [4]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
vocab_size


 !"&'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz﻿


76

In [5]:
n_embed=32

In [6]:
str_to_int={ch:i for i,ch in enumerate(chars)}
int_to_str={i:ch for i,ch in enumerate(chars)}
encode= lambda s: [str_to_int[c] for c in s]
decode= lambda l: ''.join([int_to_str[i]for i in l])

print(decode(encode('hello')))

hello


In [7]:
data=torch.tensor(encode(text),dtype=torch.long)
print(data)

tensor([75, 39, 66,  ..., 69, 61, 10])


In [8]:
n= int(0.9* len(data))
train_data= data[:n]
val_data=data[n:]

In [9]:
block_size=8
train_data[:block_size+1]

tensor([75, 39, 66, 63, 52, 69, 51, 53, 52])

In [10]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f'when i/p is {context} the target is {target}')

when i/p is tensor([75]) the target is 39
when i/p is tensor([75, 39]) the target is 66
when i/p is tensor([75, 39, 66]) the target is 63
when i/p is tensor([75, 39, 66, 63]) the target is 52
when i/p is tensor([75, 39, 66, 63, 52]) the target is 69
when i/p is tensor([75, 39, 66, 63, 52, 69]) the target is 51
when i/p is tensor([75, 39, 66, 63, 52, 69, 51]) the target is 53
when i/p is tensor([75, 39, 66, 63, 52, 69, 51, 53]) the target is 52


In [11]:
torch.manual_seed(1337)
batch_size=4 #for parallel processing by gpu
block_size=8 # maximum context length
def get_batch(split):
    #generate small batch of data of inputs x and targets y
    data= train_data if split == 'train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x= torch.stack([data[i:i+block_size]for i in ix])
    y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y


xb,yb= get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

for b in range(batch_size):# batch dimension
    for t in range(block_size):# time aspect
        context= xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is{context.tolist()} the target: {target}')
        

inputs:
torch.Size([4, 8])
tensor([[52,  1, 56, 57, 67,  1, 56, 53],
        [ 1, 51, 49, 62,  5, 68,  1, 59],
        [51, 63, 70, 53, 66, 53, 52,  1],
        [56, 53, 66, 53,  1, 57, 67,  1]])
targets:
torch.Size([4, 8])
tensor([[ 1, 56, 57, 67,  1, 56, 53, 49],
        [51, 49, 62,  5, 68,  1, 59, 53],
        [63, 70, 53, 66, 53, 52,  1, 56],
        [53, 66, 53,  1, 57, 67,  1, 49]])
when input is[52] the target: 1
when input is[52, 1] the target: 56
when input is[52, 1, 56] the target: 57
when input is[52, 1, 56, 57] the target: 67
when input is[52, 1, 56, 57, 67] the target: 1
when input is[52, 1, 56, 57, 67, 1] the target: 56
when input is[52, 1, 56, 57, 67, 1, 56] the target: 53
when input is[52, 1, 56, 57, 67, 1, 56, 53] the target: 49
when input is[1] the target: 51
when input is[1, 51] the target: 49
when input is[1, 51, 49] the target: 62
when input is[1, 51, 49, 62] the target: 5
when input is[1, 51, 49, 62, 5] the target: 68
when input is[1, 51, 49, 62, 5, 68] the targe

In [12]:
print(xb)#i/p

tensor([[52,  1, 56, 57, 67,  1, 56, 53],
        [ 1, 51, 49, 62,  5, 68,  1, 59],
        [51, 63, 70, 53, 66, 53, 52,  1],
        [56, 53, 66, 53,  1, 57, 67,  1]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Each token reads off the logits for the next token from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.position_embedding_table = nn.Embedding(100, vocab_size)
        self.lm_head = nn.Linear(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        idx = idx.to(device).long()
        tok_emb = self.token_embedding_table(idx)  # (Batch, Time, Channel)
        B, T, C = tok_emb.shape
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))  # (Time, Channel)
        x = tok_emb + pos_emb  # (B, T, C)
        logits = self.lm_head(x)  # (Batch, Time, Vocab_size)
        
        if targets is not None:
            targets = targets.to(device).view(B * T)
            logits = logits.view(B * T, C)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        
        return logits

    def generate(self, idx, max_new_tokens):
        idx = idx.to(device).long()
        for _ in range(max_new_tokens):
            logits = self(idx)  # (B, T, C)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)  # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # Append to the sequence
        return idx

# Sample input data
#vocab_size = 100
#B, T = 4, 8
#xb = torch.randint(0, vocab_size, (B, T)).to(device)
#yb = torch.randint(0, vocab_size, (B, T)).to(device)

# Initialize the model
m = BigramLanguageModel(vocab_size).to(device)

# Forward pass
logits, loss = m(xb, yb)
print(f"Logits shape: {logits.shape}")  # Should print: torch.Size([32, 100])
print(f"Loss: {loss}")  # Should print the loss value

# Generate new tokens
idx = torch.zeros(1, 1, dtype=torch.long).to(device)
generated = m.generate(idx, max_new_tokens=100)
print(f"Generated sequence: {generated}")


Logits shape: torch.Size([32, 76])
Loss: 4.784965991973877
Generated sequence: tensor([[ 0, 75,  2, 12, 54, 45,  4, 51, 31, 15, 39, 52, 22, 42, 17, 70, 45,  6,
         34, 24, 14, 32,  3, 26, 47, 36,  7,  5, 35, 20, 30, 68, 71, 44, 20, 40,
         51, 27, 65, 27, 46, 18,  7,  4, 72, 50, 21, 36, 29, 17, 61, 11,  1, 39,
         16, 35, 47, 34, 26, 67, 28,  8, 62, 57, 58, 28, 74,  3, 47, 31, 66, 50,
         69, 64, 56,  5, 56, 50, 74, 11, 56, 33,  3, 67, 12, 21, 28, 26,  1, 41,
         69, 72, 55, 67, 15, 49, 37, 34,  5, 69, 64]])


In [14]:
#training the model
#pytorch optimizer
optimizer= torch. optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size=32
for steps in range (100000):
    # sample a batch of data
    xb,yb = get_batch('train')

    # eval of loss
    logits, loss= m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.387505292892456


## Math trick for self-attention


In [17]:
#set up a means for tokens to communicate
torch.manual_seed(1337)

B,T,C = 4,8,2 # batch time and channels
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [18]:
# to get x[b,t] = avg(mean {i<=t} x[b,i])

xbow = torch.zeros((B,T,C))     #bag of words|| avg
for b in range(B):
    for t in range(T):
        xprev= x[b,:t+1] #(t,C)
        xbow[b,t]= torch.mean(xprev,0)

In [19]:
#using matrix multiplication
torch.manual_seed(1337)
a= torch.tril(torch.ones(3,3))
a= a/ torch.sum(a,1,keepdim=True)
b=torch.randint(0,10,(3,2)).float()
c=a@b
print(f"a:\n{a}\n--")
print(f"b:\n{b}\n--")
print(f"c:\n{c}\n--")

a:
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b:
tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])
--
c:
tensor([[5.0000, 7.0000],
        [3.5000, 3.5000],
        [4.0000, 3.3333]])
--


In [20]:
#version 2:

wei= torch.tril(torch.ones(T,T))
wei = wei/ wei.sum(1, keepdim=True)
xbow2= wei @ x # (T,T) @ (B,T,C)---->(b,t,c)
#------>torch.allclose(xbow,xbow2)

In [21]:
#version 3: use softmax

tril=torch.tril(torch.ones(T,T))
wei= torch.zeros((T,T))
wei= wei.masked_fill(tril==0, float('inf'))
wei=F.softmax(wei,dim=-1)
xbow3=wei@x

In [22]:
# version 4: self-attention
torch.manual_seed(1337)

B,T,C=4,8,32 # batch, time and channels
x=torch.randn(B,T,C)

#a single head's performance in self attention
head_size=16
key= nn.Linear(C,head_size, bias=False)
query= nn.Linear(C,head_size, bias=False)
value= nn.Linear(C,head_size, bias=False)
k=key(x) #(B,T,16)
q=query(x) #(B,T,16)

wei= q@ k.transpose(-2,-1) #-->(B,T,16) @ (B,16,T)-->(B,T,T))


tril=torch.tril(torch.ones(T,T))
#wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0, float('-inf'))
wei=F.softmax(wei,dim=-1)
v=value(x)
out=wei@v
#out= wei@ x
out. shape

torch.Size([4, 8, 16])

In [23]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [24]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [25]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [26]:
k.var()

tensor(1.0449)

In [27]:
q.var()

tensor(1.0700)

In [28]:
wei.var()

tensor(1.0918)

In [29]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [30]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [31]:
class LayerNorm1d: # implementing Layernorm

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) #mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalise to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [32]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [33]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))